In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
import os
import shutil

In [2]:
df2 = pd.read_csv('cxr-record-list.csv')
df4 = pd.read_csv('mimic-cxr-2.0.0-chexpert.csv')
df6 = pd.read_csv('mimic-cxr-2.0.0-metadata.csv')
merge_df = df2.merge(df4,on=['subject_id','study_id'],how='inner')
merge_df = merge_df.merge(df6[['dicom_id', 'ViewPosition']],on='dicom_id',how='inner')

In [8]:
merge_df.columns

Index(['subject_id', 'study_id', 'dicom_id', 'path', 'Atelectasis',
       'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
       'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding',
       'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax',
       'Support Devices', 'ViewPosition'],
      dtype='object')

In [3]:
def csv_to_train_val(df,img_path,disease):
    '''
    split data to train ans validation
    disease labe must be 1 for case or 0 for control, labeled by chexpert
    images are all posterior to anterior (filtered when downloading)
    choose one PA image per subject
    '''
    file_ids = [file.replace(".jpg","") for file in os.listdir(img_path)]
    df = df[df['dicom_id'].isin(file_ids)]
    
    df.loc[:,'image_path'] = img_path+ '/' + df['dicom_id'] + '.jpg'
    
    input_df = df[df[disease].isin([0,1])].drop_duplicates('subject_id')
    train_df, val_df = train_test_split(
        input_df, 
        test_size=0.2, 
        random_state=1, 
        stratify=input_df[disease]
    )
    return train_df, val_df

def csv_to_test(df,img_path,disease):
    '''
    disease labe must be 1 for case or 0 for control, labeled by radiologist
    choose one PA image per subject
    '''
    file_ids = [file.replace(".jpg","") for file in os.listdir(f'{img_path}/{disease}')]
    df = df[df['dicom_id'].isin(file_ids)]
    
    df.loc[:,'image_path'] = img_path+ f'/{disease}/' + df['dicom_id'] + '.jpg'
    test_df = df[df[disease].isin([0,1])].drop_duplicates('subject_id')

    return test_df

In [4]:
# 1. Define the custom dataset class
class disease_Dataset(Dataset):
    def __init__(self, df, disease, transform=None):
        self.df = df
        self.transform = transform
        self.disease = disease
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['image_path']).convert("RGB") #images are gray scale, but models require RGB
        label = torch.tensor(row[self.disease], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label


# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def generate_dataLoader(disease, train_df,val_df, test_df):
    # DataLoaders
    train_dataset = disease_Dataset(train_df,disease, transform)
    val_dataset = disease_Dataset(val_df,disease, transform)
    test_dataset = disease_Dataset(test_df,disease, transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    test_loader = DataLoader(test_dataset, batch_size=32)
    return train_loader, val_loader, test_loader

In [7]:
def train(model, train_loader, val_loader, lr=1e-5, weight_decay=1e-4, dropout_prob=0.5, use_scheduler=True):

    # Add dropout to the fully connected layer dynamically
    model.fc = torch.nn.Sequential(
        torch.nn.Dropout(p=dropout_prob),
        torch.nn.Linear(model.fc.in_features, 1)
    ).to(device)  # Ensure the modified layer is on the correct device

    # Define loss function and optimizer
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Optionally use a learning rate scheduler
    scheduler = None
    if use_scheduler:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.1, patience=2, verbose=True
        )

    # Training loop with early stopping
    num_epochs = 20
    patience = 3  # Number of epochs to wait for improvement
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            # Move images and labels to the specified device
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_train_loss = running_loss / len(train_loader.dataset)

        # Evaluate on the validation set
        model.eval()
        val_labels = []
        val_preds = []
        val_running_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                # Move images and labels to the specified device
                images, labels = images.to(device), labels.to(device)
                outputs = model(images).squeeze()
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * images.size(0)
                preds = torch.sigmoid(outputs).cpu().numpy()  # Move predictions to CPU for evaluation
                val_preds.extend(preds)
                val_labels.extend(labels.cpu().numpy())  # Move labels to CPU for evaluation

        epoch_val_loss = val_running_loss / len(val_loader.dataset)

        # Validation metrics
        val_binary_preds = [1 if p > 0.5 else 0 for p in val_preds]
        val_accuracy = accuracy_score(val_labels, val_binary_preds)
        val_f1 = f1_score(val_labels, val_binary_preds)
        val_auc = roc_auc_score(val_labels, val_preds)

        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Training Loss: {epoch_train_loss:.4f}")
        print(f"  Validation Loss: {epoch_val_loss:.4f}")
        print(f"  Validation Accuracy: {val_accuracy:.4f}")
        print(f"  Validation F1 Score: {val_f1:.4f}")
        print(f"  Validation AUC: {val_auc:.4f}")

        # Update the learning rate scheduler
        if use_scheduler:
            scheduler.step(epoch_val_loss)

        # Check for early stopping
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            patience_counter = 0  # Reset counter if validation loss improves
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered. Stopping training.")
                break

    return model

In [33]:
test(model_p2, test_loader)


Test Evaluation:
  Test Accuracy: 0.6792
  Test F1 Score: 0.6047
  Test AUC: 0.6571


In [6]:
def test(model, test_loader, disease):
    model.eval()
    test_labels, test_preds = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze()

            # Ensure outputs are always a batch
            if outputs.dim() == 0:  # If outputs is scalar, make it a batch
                outputs = outputs.unsqueeze(0)

            preds = torch.sigmoid(outputs).cpu().numpy()
            test_preds.extend(preds)
            test_labels.extend(labels.cpu().numpy())
    
    test_binary_preds = [1 if p > 0.5 else 0 for p in test_preds]
    test_accuracy = accuracy_score(test_labels, test_binary_preds)
    test_f1 = f1_score(test_labels, test_binary_preds)
    test_auc = roc_auc_score(test_labels, test_preds)

    print("\nTest Evaluation:")
    print(f"  Test Accuracy: {test_accuracy:.4f}")
    print(f"  Test F1 Score: {test_f1:.4f}")
    print(f"  Test AUC: {test_auc:.4f}")


In [61]:
test(model, test_loader, disease)


Test Evaluation:
  Test Accuracy: 0.7692
  Test F1 Score: 0.8052
  Test AUC: 0.8780


In [38]:
predict_models = {}
for disease in ['Pneumonia', 'Pleural Effusion','Pneumothorax']: #
    train_df, val_df = csv_to_train_val(merge_df,'images_NEW',disease) #2700
    test_df = csv_to_test(merge_df,'Testing',disease)
    train_loader, val_loader, test_loader = generate_dataLoader(disease,train_df,val_df,test_df)
    
    model = models.resnet18(pretrained=True)
    model.fc = torch.nn.Linear(model.fc.in_features, 1)
    model = model.to(device)
    print(f"=====training {disease}=====")
    model = train(model, train_loader, val_loader, lr=1e-5, weight_decay=1e-4, dropout_prob=0.5, use_scheduler=False)
    predict_models[disease] = model
    print(f"=====testing {disease}=====")
    
    test(model, test_loader, disease)
    torch.save(model.state_dict(), f"CNN_models/{disease}_model_weights.pth")

/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be

=====training Pleural Effusion=====
Epoch 1/20:
  Training Loss: 0.8540
  Validation Loss: 0.8810
  Validation Accuracy: 0.2879
  Validation F1 Score: 0.3896
  Validation AUC: 0.4893
Epoch 2/20:
  Training Loss: 0.7847
  Validation Loss: 0.7713
  Validation Accuracy: 0.4040
  Validation F1 Score: 0.4100
  Validation AUC: 0.5922
Epoch 3/20:
  Training Loss: 0.7220
  Validation Loss: 0.6778
  Validation Accuracy: 0.5354
  Validation F1 Score: 0.4458
  Validation AUC: 0.7203
Epoch 4/20:
  Training Loss: 0.6650
  Validation Loss: 0.6002
  Validation Accuracy: 0.6869
  Validation F1 Score: 0.5156
  Validation AUC: 0.7694
Epoch 5/20:
  Training Loss: 0.6246
  Validation Loss: 0.6001
  Validation Accuracy: 0.6667
  Validation F1 Score: 0.5000
  Validation AUC: 0.7696
Epoch 6/20:
  Training Loss: 0.5647
  Validation Loss: 0.5325
  Validation Accuracy: 0.7323
  Validation F1 Score: 0.5546
  Validation AUC: 0.8249
Epoch 7/20:
  Training Loss: 0.5471
  Validation Loss: 0.5092
  Validation Accurac

TypeError: iteration over a 0-d array

In [9]:
predict_models = {}
for disease in ['Pleural Effusion', 'Pneumonia', 'Pneumothorax','Atelectasis',
       'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
       'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding',
        'Pleural Other', 'Support Devices']:
    train_df, val_df = csv_to_train_val(merge_df,'images_NEW',disease) #2700
    test_df = csv_to_test(merge_df,'Testing',disease)
    train_loader, val_loader, test_loader = generate_dataLoader(disease,train_df,val_df,test_df)
    
    model = models.resnet18(pretrained=True)
    model.fc = torch.nn.Linear(model.fc.in_features, 1)
    model = model.to(device)
    print(f"=====training {disease}=====")
    model = train(model, train_loader, val_loader, lr=1e-5, weight_decay=1e-4, dropout_prob=0.5, use_scheduler=False)
    predict_models[disease] = model
    print(f"=====testing {disease}=====")
    
    test(model, test_loader, disease)
    torch.save(model.state_dict(), f"CNN_models/{disease}_model_weights.pth")

/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy o

=====training Pleural Effusion=====
Epoch 1/20:
  Training Loss: 0.6577
  Validation Loss: 0.6217
  Validation Accuracy: 0.6717
  Validation F1 Score: 0.3434
  Validation AUC: 0.6421
Epoch 2/20:
  Training Loss: 0.5913
  Validation Loss: 0.5985
  Validation Accuracy: 0.7172
  Validation F1 Score: 0.4286
  Validation AUC: 0.6729
Epoch 3/20:
  Training Loss: 0.5571
  Validation Loss: 0.5290
  Validation Accuracy: 0.7929
  Validation F1 Score: 0.5773
  Validation AUC: 0.8042
Epoch 4/20:
  Training Loss: 0.5349
  Validation Loss: 0.4890
  Validation Accuracy: 0.8283
  Validation F1 Score: 0.6136
  Validation AUC: 0.8267
Epoch 5/20:
  Training Loss: 0.4991
  Validation Loss: 0.4595
  Validation Accuracy: 0.8283
  Validation F1 Score: 0.6222
  Validation AUC: 0.8476
Epoch 6/20:
  Training Loss: 0.4762
  Validation Loss: 0.4279
  Validation Accuracy: 0.8535
  Validation F1 Score: 0.6667
  Validation AUC: 0.8525
Epoch 7/20:
  Training Loss: 0.4689
  Validation Loss: 0.4380
  Validation Accurac

/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy o

=====training Pneumonia=====
Epoch 1/20:
  Training Loss: 0.7858
  Validation Loss: 0.7724
  Validation Accuracy: 0.3907
  Validation F1 Score: 0.4695
  Validation AUC: 0.5321
Epoch 2/20:
  Training Loss: 0.7188
  Validation Loss: 0.6598
  Validation Accuracy: 0.6136
  Validation F1 Score: 0.4091
  Validation AUC: 0.5884
Epoch 3/20:
  Training Loss: 0.6650
  Validation Loss: 0.6215
  Validation Accuracy: 0.6688
  Validation F1 Score: 0.3217
  Validation AUC: 0.6208
Epoch 4/20:
  Training Loss: 0.6347
  Validation Loss: 0.6080
  Validation Accuracy: 0.7049
  Validation F1 Score: 0.3535
  Validation AUC: 0.6354
Epoch 5/20:
  Training Loss: 0.6140
  Validation Loss: 0.5827
  Validation Accuracy: 0.7028
  Validation F1 Score: 0.3204
  Validation AUC: 0.6859
Epoch 6/20:
  Training Loss: 0.6052
  Validation Loss: 0.5755
  Validation Accuracy: 0.7070
  Validation F1 Score: 0.3365
  Validation AUC: 0.6837
Epoch 7/20:
  Training Loss: 0.5909
  Validation Loss: 0.5830
  Validation Accuracy: 0.70

/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy o

=====training Pneumothorax=====
Epoch 1/20:
  Training Loss: 0.9029
  Validation Loss: 1.0286
  Validation Accuracy: 0.0392
  Validation F1 Score: 0.0755
  Validation AUC: 0.5714
Epoch 2/20:
  Training Loss: 0.8951
  Validation Loss: 1.0187
  Validation Accuracy: 0.0784
  Validation F1 Score: 0.0784
  Validation AUC: 0.3776
Epoch 3/20:
  Training Loss: 0.9254
  Validation Loss: 0.9864
  Validation Accuracy: 0.1176
  Validation F1 Score: 0.0816
  Validation AUC: 0.7449
Epoch 4/20:
  Training Loss: 0.9210
  Validation Loss: 0.9992
  Validation Accuracy: 0.0980
  Validation F1 Score: 0.0800
  Validation AUC: 0.4490
Epoch 5/20:
  Training Loss: 0.8446
  Validation Loss: 0.9518
  Validation Accuracy: 0.1569
  Validation F1 Score: 0.0851
  Validation AUC: 0.5102
Epoch 6/20:
  Training Loss: 0.8962
  Validation Loss: 0.9807
  Validation Accuracy: 0.1176
  Validation F1 Score: 0.0816
  Validation AUC: 0.6020
Epoch 7/20:
  Training Loss: 0.7902
  Validation Loss: 0.9580
  Validation Accuracy: 0

/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy o

=====training Atelectasis=====
Epoch 1/20:
  Training Loss: 0.5900
  Validation Loss: 0.4148
  Validation Accuracy: 0.9459
  Validation F1 Score: 0.9718
  Validation AUC: 0.6580
Epoch 2/20:
  Training Loss: 0.5944
  Validation Loss: 0.4093
  Validation Accuracy: 0.9324
  Validation F1 Score: 0.9650
  Validation AUC: 0.5594
Epoch 3/20:
  Training Loss: 0.5643
  Validation Loss: 0.4018
  Validation Accuracy: 0.9324
  Validation F1 Score: 0.9650
  Validation AUC: 0.4986
Epoch 4/20:
  Training Loss: 0.5306
  Validation Loss: 0.4198
  Validation Accuracy: 0.9189
  Validation F1 Score: 0.9577
  Validation AUC: 0.4087
Epoch 5/20:
  Training Loss: 0.5467
  Validation Loss: 0.4300
  Validation Accuracy: 0.9324
  Validation F1 Score: 0.9650
  Validation AUC: 0.4783
Epoch 6/20:
  Training Loss: 0.5123
  Validation Loss: 0.4332
  Validation Accuracy: 0.9189
  Validation F1 Score: 0.9577
  Validation AUC: 0.4464
Early stopping triggered. Stopping training.
=====testing Atelectasis=====


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [11]:
for disease in ['Atelectasis',
       'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
       'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding',
        'Pleural Other', 'Support Devices']:
    train_df, val_df = csv_to_train_val(merge_df,'images_NEW',disease) #2700
    test_df = csv_to_test(merge_df,'Testing',disease)
    train_loader, val_loader, test_loader = generate_dataLoader(disease,train_df,val_df,test_df)
    
    model = models.resnet18(pretrained=True)
    model.fc = torch.nn.Linear(model.fc.in_features, 1)
    model = model.to(device)
    print(f"=====training {disease}=====")
    try:
        model = train(model, train_loader, val_loader, lr=1e-5, weight_decay=1e-4, dropout_prob=0.5, use_scheduler=False)
        predict_models[disease] = model
        print(f"=====testing {disease}=====")
    
        test(model, test_loader, disease)
        torch.save(model.state_dict(), f"CNN_models/{disease}_model_weights.pth")
    except ValueError:
        print(f"ValueError {disease}")

/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/hsieh.fe/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be

=====training Atelectasis=====
Epoch 1/20:
  Training Loss: 0.6420
  Validation Loss: 0.6045
  Validation Accuracy: 0.7703
  Validation F1 Score: 0.8682
  Validation AUC: 0.3884
Epoch 2/20:
  Training Loss: 0.6073
  Validation Loss: 0.5982
  Validation Accuracy: 0.7568
  Validation F1 Score: 0.8594
  Validation AUC: 0.4870
Epoch 3/20:
  Training Loss: 0.5972
  Validation Loss: 0.5743
  Validation Accuracy: 0.7973
  Validation F1 Score: 0.8819
  Validation AUC: 0.7275
Epoch 4/20:
  Training Loss: 0.5532
  Validation Loss: 0.5709
  Validation Accuracy: 0.7568
  Validation F1 Score: 0.8594
  Validation AUC: 0.6290
Epoch 5/20:
  Training Loss: 0.5236
  Validation Loss: 0.5476
  Validation Accuracy: 0.8378
  Validation F1 Score: 0.9118
  Validation AUC: 0.4116
Epoch 6/20:
  Training Loss: 0.5336
  Validation Loss: 0.5012
  Validation Accuracy: 0.9054
  Validation F1 Score: 0.9504
  Validation AUC: 0.5855
Epoch 7/20:
  Training Loss: 0.4998
  Validation Loss: 0.4595
  Validation Accuracy: 0.

/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy o

=====training Cardiomegaly=====
Epoch 1/20:
  Training Loss: 0.8919
  Validation Loss: 0.8867
  Validation Accuracy: 0.5425
  Validation F1 Score: 0.7034
  Validation AUC: 0.5599
Epoch 2/20:
  Training Loss: 0.7760
  Validation Loss: 0.8381
  Validation Accuracy: 0.5425
  Validation F1 Score: 0.7034
  Validation AUC: 0.6114
Epoch 3/20:
  Training Loss: 0.7353
  Validation Loss: 0.7323
  Validation Accuracy: 0.5621
  Validation F1 Score: 0.7124
  Validation AUC: 0.7213
Epoch 4/20:
  Training Loss: 0.6782
  Validation Loss: 0.6739
  Validation Accuracy: 0.6013
  Validation F1 Score: 0.7313
  Validation AUC: 0.7310
Epoch 5/20:
  Training Loss: 0.6673
  Validation Loss: 0.6297
  Validation Accuracy: 0.6732
  Validation F1 Score: 0.7573
  Validation AUC: 0.7508
Epoch 6/20:
  Training Loss: 0.6283
  Validation Loss: 0.5857
  Validation Accuracy: 0.6797
  Validation F1 Score: 0.7562
  Validation AUC: 0.7821
Epoch 7/20:
  Training Loss: 0.6355
  Validation Loss: 0.5603
  Validation Accuracy: 0

/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy o

...

=====training No Finding=====
ValueError No Finding


/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy o

=====training Pleural Other=====
Epoch 1/20:
  Training Loss: 0.8072
  Validation Loss: 1.2962
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0000
  Validation AUC: 0.4000
Epoch 2/20:
  Training Loss: 0.9218
  Validation Loss: 1.2228
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0000
  Validation AUC: 0.7000
Epoch 3/20:
  Training Loss: 0.9204
  Validation Loss: 1.2246
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0000
  Validation AUC: 0.9000
Epoch 4/20:
  Training Loss: 0.8539
  Validation Loss: 1.1739
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0000
  Validation AUC: 1.0000
Epoch 5/20:
  Training Loss: 0.8858
  Validation Loss: 1.1702
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0000
  Validation AUC: 0.4000
Epoch 6/20:
  Training Loss: 0.8551
  Validation Loss: 1.1906
  Validation Accuracy: 0.0909
  Validation F1 Score: 0.0000
  Validation AUC: 0.3000
Epoch 7/20:
  Training Loss: 0.9025
  Validation Loss: 1.2290
  Validation Accuracy: 

/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/shared/centos7/anaconda3/2021.07-TF/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy o

=====training Support Devices=====
Epoch 1/20:
  Training Loss: 0.6657
  Validation Loss: 0.7019
  Validation Accuracy: 0.4340
  Validation F1 Score: 0.5588
  Validation AUC: 0.6583
Epoch 2/20:
  Training Loss: 0.6640
  Validation Loss: 0.5706
  Validation Accuracy: 0.8302
  Validation F1 Score: 0.9072
  Validation AUC: 0.3458
Epoch 3/20:
  Training Loss: 0.6321
  Validation Loss: 0.5031
  Validation Accuracy: 0.9057
  Validation F1 Score: 0.9505
  Validation AUC: 0.4542
Epoch 4/20:
  Training Loss: 0.6268
  Validation Loss: 0.4915
  Validation Accuracy: 0.8868
  Validation F1 Score: 0.9400
  Validation AUC: 0.2667
Epoch 5/20:
  Training Loss: 0.5965
  Validation Loss: 0.4614
  Validation Accuracy: 0.9057
  Validation F1 Score: 0.9505
  Validation AUC: 0.4500
Epoch 6/20:
  Training Loss: 0.5712
  Validation Loss: 0.4682
  Validation Accuracy: 0.8868
  Validation F1 Score: 0.9400
  Validation AUC: 0.2500
Epoch 7/20:
  Training Loss: 0.5747
  Validation Loss: 0.4905
  Validation Accuracy

In [62]:
# model = torch.load("model.pth")
# model = model.to(device)  # Move to GPU/CPU as needed
# model.eval()  # Set the model to evaluation mode
